In [1]:
import pandas as pd 
pd.set_option('display.max_columns', None)
import zipfile
from zipfile import ZipFile
import json
import numpy 
from os import listdir 

In [2]:
bills = pd.read_csv('data_store/dataframes/full_congress_level_df8.csv')
bills['major_label 1'].fillna(404, inplace=True)
bills['major_label 2'].fillna(404, inplace=True)
bills['major_label 3'].fillna(404, inplace=True)
bills['major_label 1'] = bills['major_label 1'].astype(int)
bills['major_label 2'] = bills['major_label 2'].astype(int)
bills['major_label 3'] = bills['major_label 3'].astype(int)

legislators = pd.read_csv('all_legislators.csv')
legislators['thomas_id'].fillna(0, inplace=True)
legislators['thomas_id'] = legislators['thomas_id'].astype(int) 

files = pd.read_csv('file_names.csv')

/home/william/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (2,3,4,7,8,9,12,16,17,18,20,21,24,25,26,29,37,38,39,42,45,46,47,50,53,54,55,58,61,62,63,66,69,70,71,74,77,78,79,82,85,86,87,90,93,94,95,98,101,102,103,106,109,110,111,114,115,116,117,144,145,146,150,151,152,154,155,158,159,160,163,166,167,168,170,171,174,175,176,178,179,182,183,184,187,194,195,196,200,201,202,206,208,211,212,213,215,216,221,222,223,226,229,230,231,234) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
def thomas_cleaner(df):
    df['thomas_id'] = df['thomas_id'].astype(str)
    results_list = []
    for string in df['thomas_id']:
        string_len = len(string)
        preceeding_zeroes = 5 - string_len
        results_list.append(str((preceeding_zeroes * '0') + string))
    df['thomas_id'] = results_list
    return df   

In [4]:
legislators = thomas_cleaner(legislators)

## Function to compute bill ideology from average ideologies of each bill's cosponsors in each bill's three major subject areas 

In [1]:
def bill_ideology_compiler(file_df, bill_df, legislator_df, congress_list): 
    bill_ids = []
    cosponsor_counts = []
    major_label_1_scores = []
    major_label_2_scores = []
    major_label_3_scores = []
    no_ideology_records = []
    thomas_strings = []
    bioguide_ids = []
    for i in range(len(legislators['thomas_id'])):
        thomas_strings.append(str(legislators['thomas_id'][i]))
        bioguide_ids.append(str(legislators['bioguide_id'][i]))
    thomas_dict = {'thomas': thomas_strings, 'bioguide': bioguide_ids}                    
    if congress_list == 'ALL':    
        congress_numbers = list(range(93, 99)) + list(range(101, 117))
    else:
        congress_numbers = congress_list
    for congress_number in congress_numbers:
        congress_bills = bill_df[bill_df[
            'congress'] == congress_number].reset_index(drop=True)
        sorted_bills = congress_bills.sort_values(['congress', 'bill_type', 'number'])
        sorted_bills = sorted_bills.reset_index(drop=True)
        sorted_bills = sorted_bills[['bill_id', 'major_label 1', 'major_label 2',
                                     'major_label 3']]
        running_bill_count = 0 
        thomas_ids = []
        total_bill_count = sorted_bills['bill_id'].nunique()
        ideology_files = listdir('data_store/member_ideology/')
        with zipfile.ZipFile('bill_metadata/' + str(congress_number) + '.zip', 'r') as z:
            for file in file_df['file_name'][
                file_df['congress'] == congress_number].reset_index(drop=True):
                with z.open(file) as f:
                    data = f.read()
                    d = json.loads(data.decode('utf-8'))
                    if 'cosponsors' in d.keys():
                        bill_cosponsors = d['cosponsors']
                        congress_number = int(d['congress'])
                        bill_id = d['bill_id']
                        cosponsor_counts.append(len(bill_cosponsors)) 
                        bill_ids.append(bill_id)
                        print ('Congress: ', congress_number,
                              'bill_id: ', bill_id, 'Cosponsors: ', len(bill_cosponsors),
                               '\n')
                        if len(bill_cosponsors) > 0:
                            bill_data = sorted_bills[sorted_bills['bill_id'] == 
                                                     bill_id].reset_index(drop=True)
                            label_1 = str(bill_data['major_label 1'].item())
                            label_2 = str(bill_data['major_label 2'].item())
                            label_3 = str(bill_data['major_label 3'].item())
                            feature_1 = 'topic_' + label_1 
                            feature_2 = 'topic_' + label_2
                            feature_3 = 'topic_' + label_3 
                            feature_1_nan = 0
                            feature_2_nan = 0
                            feature_3_nan = 0
                            print ('feature 1: ',feature_1)
                            print ('feature 2: ',feature_2)
                            print ('feature 3: ',feature_3)
                            if feature_1 == 'topic_404':
                                feature_1_nan +=1
                            else:
                                pass
                            if feature_2 == 'topic_404':
                                feature_2_nan +=1
                            else:
                                pass
                            if feature_3 == 'topic_404':
                                feature_3_nan +=1
                            else:
                                pass
                            if congress_number > 112:
                                label_1_score = 0
                                label_2_score = 0
                                label_3_score = 0
                                for i in range(len(bill_cosponsors)):
                                    member_id = bill_cosponsors[i]['bioguide_id']
                                    member_file = str(member_id)+'_ideology_df.csv'
                                    directory = 'data_store/member_ideology/'
                                    full_path = directory + member_file
                                    if member_file in ideology_files:
                                        member_df0 = pd.read_csv(full_path)
                                        merged_df = pd.merge(sorted_bills, member_df0,
                                                            on='bill_id', how='left')
                                        topic_cols = list(merged_df.columns[-21:])
                                        for col in topic_cols:
                                            merged_df[col] = merged_df[col].ffill()
                                        for col in topic_cols:
                                            merged_df[col] = merged_df[col].bfill()
                                        member_df = merged_df[
                                            merged_df['bill_id'] == bill_id].reset_index(
                                            drop=True)
                                        if len(member_df) > 0:
                                            member_df1 = member_df.iloc[0]
                                            if feature_1_nan == 0:
                                                member_score_1 = member_df1[
                                                    feature_1].item()
                                                label_1_score += member_score_1
                                            else:
                                                pass
                                            if feature_2_nan == 0:
                                                member_score_2 = float(member_df1[
                                                    feature_2].item())
                                                label_2_score += member_score_2
                                            else:
                                                pass
                                            if feature_3_nan == 0:
                                                member_score_3 = float(member_df1[
                                                    feature_3].item())
                                                label_3_score += member_score_3
                                            else:
                                                pass
                                        else:
                                            pass
                                    else:
                                        pass 
                                major_label_1_scores.append(
                                    (label_1_score / len(bill_cosponsors)))
                                major_label_2_scores.append(
                                    (label_2_score / len(bill_cosponsors)))
                                major_label_3_scores.append(
                                    (label_3_score / len(bill_cosponsors)))
                                running_bill_count +=1     
                                print ('label 1 score: ', major_label_1_scores[-1])
                                print ('label 2 score: ', major_label_2_scores[-1])
                                print ('label 3 score: ', major_label_3_scores[-1])
                                print ('Congress ', congress_number,
                                       ' bills analyzed: ',
                                       ((running_bill_count/total_bill_count)*100), '%')
                            else:
                                for i in range(len(bill_cosponsors)):
                                    thomas = bill_cosponsors[i]['thomas_id']
                                    label_1_score = 0
                                    label_2_score = 0
                                    label_3_score = 0
                                    if thomas in thomas_strings:
                                        index = thomas_strings.index(thomas)
                                        bioguide = bioguide_ids[index]
                                        member_file = str(bioguide)+'_ideology_df.csv'
                                        directory = 'data_store/member_ideology/'
                                        full_path = directory + member_file
                                        if member_file in ideology_files:
                                            member_df0 = pd.read_csv(full_path)
                                            merged_df = pd.merge(sorted_bills, member_df0,
                                                                 on='bill_id', how='left')
                                            topic_cols = list(merged_df.columns[-21:])
                                            for col in topic_cols:
                                                merged_df[col] = merged_df[col].ffill()
                                            for col in topic_cols:
                                                merged_df[col] = merged_df[col].bfill()
                                            member_df = merged_df[
                                                merged_df['bill_id'] == bill_id].reset_index(
                                                drop=True)
                                            if len(member_df) > 0:
                                                member_df1 = member_df.iloc[0]
                                                if feature_1_nan == 0:
                                                    member_score_1 = float(member_df1[
                                                        feature_1].item())
                                                    label_1_score += member_score_1
                                                else:
                                                    pass
                                                if feature_2_nan == 0:
                                                    member_score_2 = float(member_df1[
                                                        feature_2].item())
                                                    label_2_score += member_score_2
                                                else:
                                                    pass
                                                if feature_3_nan == 0:
                                                    member_score_3 = float(member_df1[
                                                        feature_3].item())
                                                    label_3_score += member_score_3
                                                else:
                                                    pass
                                            else:
                                                pass
                                        else:
                                            pass
                                    else:
                                        pass 
                                major_label_1_scores.append(
                                    (label_1_score / len(bill_cosponsors)))
                                major_label_2_scores.append(
                                    (label_2_score / len(bill_cosponsors)))
                                major_label_3_scores.append(
                                    (label_3_score / len(bill_cosponsors)))
                                running_bill_count +=1
                                print ('label 1 score: ', major_label_1_scores[-1])
                                print ('label 2 score: ', major_label_2_scores[-1])
                                print ('label 3 score: ', major_label_3_scores[-1])
                                print ('Congress ', congress_number,
                                       ' bills analyzed: ',
                                       ((running_bill_count/total_bill_count)*100), '%')
                            
                        else:
                            major_label_1_scores.append(0)
                            major_label_2_scores.append(0)
                            major_label_3_scores.append(0)
                    else:
                        major_label_1_scores.append(0)
                        major_label_2_scores.append(0)
                        major_label_3_scores.append(0)
                            
    data_dict = {'bill_id': bill_ids, 'cosponsors': cosponsor_counts,
                 'major_label_1 ideology': major_label_1_scores,
                 'major_label_2 ideology': major_label_2_scores,
                 'major_label_3 ideology': major_label_3_scores}
    output_df = pd.DataFrame(data_dict)
    
    return output_df  

In [8]:
len(full_ideology)

238243

In [7]:
full_ideology.to_csv('updated_bill_ideology_scores.csv', index=False)